In [1]:
from odps import ODPS
from odps import options
from odps.df import DataFrame
import pandas as pd
import numpy as np

pd.set_option('display.max_rows',None)

ACCESS_ID = 'LTAI5t9uwJrh5eJ7Q5E37D1s'
SECRET_ACCESS_KEY = 'NCFHOAnvqfnTrpypgR4b3cNawP8fnB'
ODPS_PROJECT = 'OpenDigger_prod_dev'
ODPS_ENDPOINT = 'http://service.cn-shanghai.maxcompute.aliyun.com/api'

o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
         project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False
# options.read_timeout = 10000000

users = DataFrame(o.get_table('ods_github_users'))
print(users.dtypes)

github_log = DataFrame(o.get_table('ods_github_log'))
print(github_log.dtypes)

odps.Schema {
  login                     string        
  created_at                date          
  database_id               int64         
  location                  string        
  company                   string        
  bio                       string        
  is_employee               boolean       
  email                     string        
  infoname                  string        
  followers                 string        
  following                 string        
  time                      date          
  name                      string        
  lastupdatedat             date          
  nextupdateat              date          
}

odps.Schema {
  id                                                                          string                  
  type                                                                        string                  
  action                                                                      string                  
  actor_id     

In [8]:
sql = '''
    select type, count(repo_id), repo_id from ods_github_log
    where pt='20151001'
    and type in ('PullRequestEvent','WatchEvent','ForkEvent','IssueCommentEvent')
    group by type, repo_id;
'''

result = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})
with open('data\count.txt', 'w') as f:
    with result.open_reader() as reader:
        for record in reader:
            type = record['type']
            count = record['_c1']
            repo_id = record['repo_id']       
            f.write('type: {type}, repo_id: {repo_id}, count: {count}\n'.format(
                                                        type=type,
                                                        repo_id=repo_id,
                                                        count=count))    

In [6]:
graph_dict = {}
graph_dict['PullRequestEvent'] = {}
graph_dict['WatchEvent'] = {}
graph_dict['ForkEvent'] = {}
graph_dict['IssueCommentEvent'] = {}
sql = '''
    select type, repo_id, actor_id
    from ods_github_log
    where pt='20151001'
        and type in ('PullRequestEvent','WatchEvent','ForkEvent','IssueCommentEvent')
    group by type, repo_id, actor_id;
'''
result = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})
with result.open_reader() as reader:
    for record in reader:
        type = record['type']
        actor_id = record['actor_id']
        repo_id = record['repo_id']
        if actor_id not in graph_dict[type]:
            graph_dict[type][actor_id] = []
        graph_dict[type][actor_id].append(str(repo_id))

# print(graph_dict)

with open('data\PullRequestEvent.txt', 'w') as f:
    for key in graph_dict['PullRequestEvent']:
        if len(graph_dict['PullRequestEvent'][key]) < 2:
            continue
        f.write('actor_id: {actor_id}\nrepo_id: {list}\n'.format(actor_id=key, list=' '.join(graph_dict['PullRequestEvent'][key])))

with open('data\WatchEvent.txt', 'w') as f:
    for key in graph_dict['WatchEvent']:
        if len(graph_dict['WatchEvent'][key]) < 2:
            continue
        f.write('actor_id: {actor_id}\nrepo_id: {list}\n'.format(actor_id=key, list=' '.join(graph_dict['WatchEvent'][key])))

with open('data\ForkEvent.txt', 'w') as f:
    for key in graph_dict['ForkEvent']:
        if len(graph_dict['ForkEvent'][key]) < 2:
            continue
        f.write('actor_id: {actor_id}\nrepo_id: {list}\n'.format(actor_id=key, list=' '.join(graph_dict['ForkEvent'][key])))

with open('data\IssueCommentEvent.txt', 'w') as f:
    for key in graph_dict['IssueCommentEvent']:
        if len(graph_dict['IssueCommentEvent'][key]) < 2:
            continue
        f.write('actor_id: {actor_id}\nrepo_id: {list}\n'.format(actor_id=key, list=' '.join(graph_dict['IssueCommentEvent'][key])))